In [1]:
!pip install -r "../requirements.txt"


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [23]:
from one.api import ONE
import pandas as pd
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from one.api import ONE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [3]:
ONE.setup(base_url='https://openalyx.internationalbrainlab.org', silent=True)
one = ONE(password='international')

In [4]:
## Load Data

In [5]:
one = ONE(base_url='https://openalyx.internationalbrainlab.org')

print(one.search_terms())

('dataset', 'date_range', 'laboratory', 'number', 'projects', 'subject', 'task_protocol', 'dataset_qc_lte')


In [6]:
help(one.search)

Help on method search in module one.api:

search(details=False, query_type=None, **kwargs) method of one.api.OneAlyx instance
    Searches sessions matching the given criteria and returns a list of matching eids.

    For a list of search terms, use the method

        one.search_terms(query_type='remote')

    For all search parameters, a single value or list may be provided.  For `dataset`,
    the sessions returned will contain all listed datasets.  For the other parameters,
    the session must contain at least one of the entries.

    For all but `date_range` and `number`, any field that contains the search string is
    returned.  Wildcards are not permitted, however if wildcards property is True, regular
    expressions may be used (see notes and examples).

    Parameters
    ----------
    dataset : str
        A (partial) dataset name. Returns sessions containing matching datasets.
        A dataset matches if it contains the search string e.g. 'wheel.position' matches
      

In [7]:
# All datasets in the database
dsets = one.list_datasets()
dsets

['alf/_ibl_trials.table.pqt',
 'alf/_ibl_trials.repNum.npy',
 'alf/probe00/pykilosort/#2024-05-06#/spikes.times.npy',
 'alf/probe00/pykilosort/#2024-05-06#/clusters.depths.npy',
 'raw_ephys_data/probe00/_spikeglx_ephysData_g0_t0.imec0.lf.meta',
 'alf/probe00/pykilosort/spikes.times.npy',
 'alf/probe00/pykilosort/#2024-05-06#/channels.brainLocationIds_ccf_2017.npy',
 'alf/rightROIMotionEnergy.position.npy',
 'raw_ephys_data/probe00/_spikeglx_ephysData_g0_t0.imec0.ap.ch',
 'raw_ephys_data/probe00/_iblqc_ephysChannels.apRMS.npy',
 'alf/probe00/pykilosort/#2024-05-06#/spikes.depths.npy',
 'alf/_ibl_rightCamera.features.pqt',
 'raw_ephys_data/probe00/_iblqc_ephysSpectralDensityLF.power.npy',
 'raw_ephys_data/_spikeglx_sync.channels.npy',
 'alf/_ibl_wheel.position.npy',
 'alf/probe00/pykilosort/#2024-05-06#/clusters.uuids.csv',
 'alf/_ibl_leftCamera.times.npy',
 'raw_ephys_data/_spikeglx_ephysData_g0_t0.nidq.cbin',
 'alf/probe00/pykilosort/#2024-05-06#/spikes.clusters.npy',
 'alf/leftROIMoti

In [27]:
one.describe_dataset('trials.table');

A table of trial events, one per trial. Events include: intervals, goCue_times, response_times, choice, stimOn_times, contrastLeft, contrastRight, feedback_times, feedbackType, rewardVolume, probabilityLeft, firstMovement_times


In [29]:
eids, details = one.search(dataset='trials.table', details=True)
# eids, details = one.search(dataset='alf/_ibl_trials.feedbackType.npy', details=True)
pprint(eids)
pprint(len(eids))
# pprint(details)
# pprint(len(details))

['ebce500b-c530-47de-8cb1-963c552703ea',
 'a7eba2cf-427f-4df9-879b-e53e962eae18',
 '3a3ea015-b5f4-4e8b-b189-9364d1fc7435',
 '5ae68c54-2897-4d3a-8120-426150704385',
 'd85c454e-8737-4cba-b6ad-b2339429d99b',
 '11163613-a6c9-4975-9586-84dc00481547',
 'de905562-31c6-4c31-9ece-3ee87b97eab4',
 '15b69921-d471-4ded-8814-2adad954bcd8',
 'e6594a5b-552c-421a-b376-1a1baa9dc4fd',
 '4e560423-5caf-4cda-8511-d1ab4cd2bf7d',
 'c46b8def-620d-4d70-9397-be5694255f73',
 '6899a67d-2e53-4215-a52a-c7021b5da5d4',
 '3f71aa98-08c6-4e79-b4c8-00eae4f03eff',
 'a1782f4f-86b0-480c-a7f2-3d8f1ab482ab',
 '97c1d08d-57b4-4595-8052-91eb84ebfd74',
 '35eeb752-8f4f-4040-9714-ba0f5b7ccdfe',
 '004d8fd5-41e7-4f1b-a45b-0d4ad76fe446',
 'ca4ecb4c-4b60-4723-9b9e-2c54a6290a53',
 '7dd27c2f-9544-448d-918d-be0f9837b0e0',
 'caa5dddc-9290-4e27-9f5e-575ba3598614',
 '642c97ea-fe89-4ec9-8629-5e492ea4019d',
 'e6bdb1f4-b0bf-4451-8f23-4384f2102f91',
 '2584ce3c-db10-4076-89cb-5d313138dd38',
 '1dbba733-24a4-4400-9436-53f1bd8428e8',
 'e6de6c35-1508-

In [14]:
# eids = one.search(subject='NYU-21', dataset=['alf/_ibl_trials.feedbackType.npy'])
# pprint(eids)

In [42]:
# Initialize an empty list to hold DataFrames
dataframes = []

i = 25
eids_subset = eids[:i]
# Iterate over each eid and load trials data
for eid in eids_subset:
    try:
        # Load the trials dataset for the session
        trials = one.load_object(eid, 'trials', collection='alf')
        
        # Convert to DataFrame
        df = trials.to_df()
        
        # Add an identifier column for the session
        df['session_id'] = eid
        
        # Append the DataFrame to the list
        dataframes.append(df)
        
        # Optional: print the shape and head of the DataFrame
        print(f"Session: {eid}, Shape: {df.shape}")
        print(df.head())
        
    except Exception as e:
        print(f"Error loading trials for session {eid}: {e}")

# Concatenate all DataFrames into a single DataFrame
all_trials_df = pd.concat(dataframes, ignore_index=True)

# Check the shape of the aggregated DataFrame
print(all_trials_df.shape)
all_trials_df.head()

Session: ebce500b-c530-47de-8cb1-963c552703ea, Shape: (569, 16)
   goCueTrigger_times  stimOff_times  goCue_times  response_times  choice  \
0           13.648230      15.156255    13.649192       14.099033    -1.0   
1           16.858861      18.258806    16.859900       17.199067     1.0   
2           19.872914      21.272799    19.873893       20.212920    -1.0   
3           23.692570      25.072820    23.693512       24.023372    -1.0   
4           27.242310      28.575402    27.243189       27.521317    -1.0   

   stimOn_times  contrastLeft  contrastRight  probabilityLeft  feedback_times  \
0     13.739751           NaN          1.000              0.5       14.099118   
1     16.858734          0.25            NaN              0.5       17.199169   
2     19.872693           NaN          0.250              0.5       20.213029   
3     23.692446           NaN          0.125              0.5       24.023482   
4     27.242123           NaN          0.250              0.5       

/Users/guillemcomerma/Downloads/ONE/openalyx.internationalbrainlab.org/churchlandlab_ucla/Subjects/MFD_07/2023-08-29/001/alf/_ibl_trials.goCueTrigger_times.npy: 100%|██████████| 5.89k/5.89k [00:00<00:00, 20.3kB/s]
/Users/guillemcomerma/Downloads/ONE/openalyx.internationalbrainlab.org/churchlandlab_ucla/Subjects/MFD_07/2023-08-29/001/alf/_ibl_trials.table.pqt: 100%|██████████| 57.8k/57.8k [00:00<00:00, 158kB/s]
/Users/guillemcomerma/Downloads/ONE/openalyx.internationalbrainlab.org/churchlandlab_ucla/Subjects/MFD_07/2023-08-29/001/alf/_ibl_trials.stimOff_times.npy: 100%|██████████| 5.89k/5.89k [00:00<00:00, 21.2kB/s]


Session: c46b8def-620d-4d70-9397-be5694255f73, Shape: (720, 16)
   goCueTrigger_times  stimOff_times  goCue_times  response_times  choice  \
0           20.505216      22.723000    20.505969       21.658332     1.0   
1           26.257198      34.907326    26.258012       33.839415     1.0   
2           36.789559      38.805971    36.790355       37.742272     1.0   
3           44.290677      46.405980    44.291373       45.330995    -1.0   
4           51.590592      55.305804    51.591375       53.230216    -1.0   

   stimOn_times  contrastLeft  contrastRight  probabilityLeft  feedback_times  \
0     20.572863         1.000            NaN              0.5       21.658431   
1     26.257012         0.125            NaN              0.5       33.839523   
2     36.789421         1.000            NaN              0.5       37.742368   
3     44.290539           NaN         0.0625              0.5       45.331111   
4     51.590442         0.250            NaN              0.5       

/Users/guillemcomerma/Downloads/ONE/openalyx.internationalbrainlab.org/churchlandlab_ucla/Subjects/MFD_06/2023-08-29/001/alf/_ibl_trials.stimOff_times.npy: 100%|██████████| 3.34k/3.34k [00:00<00:00, 12.8kB/s]
/Users/guillemcomerma/Downloads/ONE/openalyx.internationalbrainlab.org/churchlandlab_ucla/Subjects/MFD_06/2023-08-29/001/alf/_ibl_trials.goCueTrigger_times.npy: 100%|██████████| 3.34k/3.34k [00:00<00:00, 10.7kB/s]
/Users/guillemcomerma/Downloads/ONE/openalyx.internationalbrainlab.org/churchlandlab_ucla/Subjects/MFD_06/2023-08-29/001/alf/_ibl_trials.table.pqt: 100%|██████████| 36.4k/36.4k [00:00<00:00, 93.5kB/s]


Session: 6899a67d-2e53-4215-a52a-c7021b5da5d4, Shape: (402, 16)
   stimOff_times  goCueTrigger_times  goCue_times  response_times  choice  \
0      13.889670           12.490990    12.491731       12.835295     1.0   
1      19.806466           18.407982    18.408726       18.739088     1.0   
2      23.957455           22.274544    22.275408       22.898153     1.0   
3      28.624097           27.273319    27.274087       27.563123    -1.0   
4      33.089724           30.306366    30.307111       32.036091     1.0   

   stimOn_times  contrastLeft  contrastRight  probabilityLeft  feedback_times  \
0     12.589655        0.2500            NaN              0.5       12.835400   
1     18.407793        0.1250            NaN              0.5       18.739196   
2     22.274441        0.1250            NaN              0.5       22.898251   
3     27.273187           NaN           0.25              0.5       27.563227   
4     30.306178        0.0625            NaN              0.5       

...

/Users/guillemcomerma/Downloads/ONE/openalyx.internationalbrainlab.org/hausserlab/Subjects/PL050/2023-06-12/001/alf/_ibl_trials.table.pqt: 100%|██████████| 51.2k/51.2k [00:00<00:00, 145kB/s]
/Users/guillemcomerma/Downloads/ONE/openalyx.internationalbrainlab.org/hausserlab/Subjects/PL050/2023-06-12/001/alf/_ibl_trials.stimOff_times.npy: 100%|██████████| 5.09k/5.09k [00:00<00:00, 18.8kB/s]
/Users/guillemcomerma/Downloads/ONE/openalyx.internationalbrainlab.org/hausserlab/Subjects/PL050/2023-06-12/001/alf/_ibl_trials.goCueTrigger_times.npy: 100%|██████████| 5.09k/5.09k [00:00<00:00, 19.0kB/s]


Session: 1dbba733-24a4-4400-9436-53f1bd8428e8, Shape: (620, 16)
   stimOff_times  goCueTrigger_times  goCue_times  response_times  choice  \
0      99.660266           97.846886    97.847714       98.571387     1.0   
1     104.893776          103.549292   103.549977      103.818292     1.0   
2     109.149517          107.832987   107.833882      108.059186     1.0   
3     112.665899          111.276770   111.277504      111.584369    -1.0   
4     115.726726          114.293456   114.294136      114.638756     1.0   

   stimOn_times  contrastLeft  contrastRight  probabilityLeft  feedback_times  \
0     97.943671        0.2500            NaN              0.5       98.571475   
1    103.549111        0.1250            NaN              0.5      103.818384   
2    107.832782        0.1250            NaN              0.5      108.059293   
3    111.276638           NaN           0.25              0.5      111.584474   
4    114.293336        0.0625            NaN              0.5      1

/Users/guillemcomerma/Downloads/ONE/openalyx.internationalbrainlab.org/hausserlab/Subjects/PL050/2023-06-11/001/alf/_ibl_trials.goCueTrigger_times.npy: 100%|██████████| 4.45k/4.45k [00:00<00:00, 15.8kB/s]
/Users/guillemcomerma/Downloads/ONE/openalyx.internationalbrainlab.org/hausserlab/Subjects/PL050/2023-06-11/001/alf/_ibl_trials.table.pqt: 100%|██████████| 46.0k/46.0k [00:00<00:00, 116kB/s]
/Users/guillemcomerma/Downloads/ONE/openalyx.internationalbrainlab.org/hausserlab/Subjects/PL050/2023-06-11/001/alf/_ibl_trials.stimOff_times.npy: 100%|██████████| 4.45k/4.45k [00:00<00:00, 16.1kB/s]

Session: e6de6c35-1508-4471-b7c3-f12a5c7a6d39, Shape: (540, 16)
   goCueTrigger_times  stimOff_times  goCue_times  response_times  choice  \
0           93.590806      95.078891    93.591573       94.033908    -1.0   
1           98.402402      99.678631    98.403159       98.622201    -1.0   
2          103.702482     105.112021   103.703429      104.046982    -1.0   
3          107.835173     109.252007   107.836015      108.174672     1.0   
4          115.078777     116.378428   115.079558      115.307477     1.0   

   stimOn_times  contrastLeft  contrastRight  probabilityLeft  feedback_times  \
0     93.678931           NaN         1.0000              0.5       94.033996   
1     98.402225           NaN         1.0000              0.5       98.622303   
2    103.702362           NaN         0.0625              0.5      104.047094   
3    107.835049         0.125            NaN              0.5      108.174781   
4    115.078591         0.250            NaN              0.5      1

,goCueTrigger_times,stimOff_times,goCue_times,response_times,choice,stimOn_times,contrastLeft,contrastRight,probabilityLeft,feedback_times,feedbackType,rewardVolume,firstMovement_times,intervals_0,intervals_1,session_id
0,13.648230,15.156255,13.649192,14.099033,-1.0,13.739751,NaN,1.000,0.5,14.099118,1.0,1.5,13.976156,12.933124,15.656348,ebce500b-c530-47de-8cb1-963c552703ea
1,16.858861,18.258806,16.859900,17.199067,1.0,16.858734,0.25,NaN,0.5,17.199169,1.0,1.5,16.983156,16.302151,18.758896,ebce500b-c530-47de-8cb1-963c552703ea
2,19.872914,21.272799,19.873893,20.212920,-1.0,19.872693,NaN,0.250,0.5,20.213029,1.0,1.5,19.987156,19.380104,21.772851,ebce500b-c530-47de-8cb1-963c552703ea
3,23.692570,25.072820,23.693512,24.023372,-1.0,23.692446,NaN,0.125,0.5,24.023482,1.0,1.5,23.805156,22.426161,25.572885,ebce500b-c530-47de-8cb1-963c552703ea
4,27.242310,28.575402,27.243189,27.521317,-1.0,27.242123,NaN,0.250,0.5,27.521431,1.0,1.5,27.313156,26.182586,29.075456,ebce500b-c530-47de-8cb1-963c552703ea


In [43]:
all_trials_df.shape

(15381, 16)

In [44]:
# Check contrast columns
contrast_columns = all_trials_df[['contrastLeft', 'contrastRight']]
contrast_columns

,contrastLeft,contrastRight
0,NaN,1.0000
1,0.250,NaN
2,NaN,0.2500
3,NaN,0.1250
4,NaN,0.2500
...,...,...
15376,NaN,0.0000
15377,NaN,0.2500
15378,0.125,NaN
15379,NaN,1.0000


In [45]:
# Check how many times both columns are NaN
both_nan_count = contrast_columns.isna().all(axis=1).sum()
both_nan_count

0

In [46]:
# select constrastLeft and if not contrastRight
all_trials_df['contrast'] = all_trials_df['contrastLeft'].fillna(all_trials_df['contrastRight'])
all_trials_df['contrast'].value_counts()

contrast
1.0000    3504
0.0625    3399
0.2500    3369
0.1250    3328
0.0000    1781
Name: count, dtype: int64

In [47]:
all_trials_df['feedbackType'].value_counts()

feedbackType
 1.0    12458
-1.0     2923
Name: count, dtype: int64

In [48]:
# Train model
# Split the data into training and test sets
X = all_trials_df[['contrast']] # 0 nan so no need to .fillna(0)
y = all_trials_df['feedbackType']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=35)

# Initialize and train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00       849
         1.0       0.82      1.00      0.90      3766

    accuracy                           0.82      4615
   macro avg       0.41      0.50      0.45      4615
weighted avg       0.67      0.82      0.73      4615

[[   0  849]
 [   0 3766]]


/Users/guillemcomerma/Developer/nma/nma_cn_project/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/guillemcomerma/Developer/nma/nma_cn_project/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/guillemcomerma/Developer/nma/nma_cn_project/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

In [49]:
all_trials_df['rewardVolume'].shape

(15381,)

In [50]:
all_trials_df['rewardVolume'].value_counts()

rewardVolume
1.5    12458
0.0     2923
Name: count, dtype: int64

In [51]:
all_trials_df.columns

Index(['goCueTrigger_times', 'stimOff_times', 'goCue_times', 'response_times',
       'choice', 'stimOn_times', 'contrastLeft', 'contrastRight',
       'probabilityLeft', 'feedback_times', 'feedbackType', 'rewardVolume',
       'firstMovement_times', 'intervals_0', 'intervals_1', 'session_id',
       'contrast'],
      dtype='object')

In [52]:
if 'rewardVolume' in all_trials_df.columns:
    print("Column 'rewardVolume' exists.")
else:
    print("Column 'rewardVolume' does not exist.")

Column 'rewardVolume' exists.


In [53]:
# Inclue more features
# Split the data into training and test sets
X = all_trials_df[['contrast', 'rewardVolume']]  # Add more features as needed
y = all_trials_df['feedbackType']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=35)

# Initialize and train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

        -1.0       1.00      1.00      1.00       849
         1.0       1.00      1.00      1.00      3766

    accuracy                           1.00      4615
   macro avg       1.00      1.00      1.00      4615
weighted avg       1.00      1.00      1.00      4615

[[ 849    0]
 [   0 3766]]
